# EXTRACCION, TRANSFORMACIÓN Y LIMPIEZA 

## Base de datos: Movies

Se utiliza la libreria de pandas para dar inicio a la estracción y tratamiento de los datos.

In [1]:
import pandas as pd
import numpy as np
import json
import ast
import os

Inicio la extracción, primero del detaset "movies" y luego del dataset "credits" para despues unirlos. 

In [2]:
# Extraigo el dataset "movies" y reviso el tamaño
df_movies = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')
df_movies.shape

C:\Users\anavi\AppData\Local\Temp\ipykernel_15588\131553651.py:2: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df_movies = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')


(45466, 24)

In [3]:
# Extraigo los dataset "credits" y reviso el tamaño
df_credits = pd.read_csv('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\credits.csv')
df_credits.shape

(45476, 3)

In [4]:
# Quiero saber el tamaño de mi dataset para reducirlo hasta llegar a 100 megas
tamaño1 = os.path.getsize('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\movies_dataset.csv')
tamaño2 = os.path.getsize('C:\\Users\\anavi\\OneDrive\\Escritorio\\Henry\\P1 individual\\Dataset original\\credits.csv')

print(f"Tamaño movies: {tamaño1 / (1024 * 1024):.2f} MB y tamaño credits: {tamaño2 / (1024 * 1024):.2f} MB")

Tamaño movies: 32.85 MB y tamaño credits: 181.12 MB


Como veo que el dataset credits es mucho más grande que el dataset movies entonces empiezo limpiandolo para reducirlo al máximo.

In [5]:
df_credits[df_credits.duplicated(keep=False)].shape # Verifico si hay datos duplicados para eliminarlos posteriormente

(73, 3)

In [6]:
df_credits = df_credits.drop_duplicates() # Elimino los datos duplicados en mi dataset original
df_credits[df_credits.duplicated(keep=False)].shape #Verifico que hayan quedado eliminados

(0, 3)

In [8]:
df_credits.columns # Verifico que columnas existen en mi data set

Index(['cast', 'crew', 'id'], dtype='object')

In [10]:
df_credits.head()

,cast,crew,id
0,"[{'cast_id': 14, 'character': 'Woody (voice)',...","[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'cast_id': 1, 'character': 'Alan Parrish', '...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'cast_id': 2, 'character': 'Max Goldman', 'c...","[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'cast_id': 1, 'character': ""Savannah 'Vannah...","[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'cast_id': 1, 'character': 'George Banks', '...","[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Mi dataset tiene dos columas. La primera contiene el elenco "cast" y la segunda contiene los nombres del equipo "crew". Voy a empezar por depurar la columna "cast" y dejar unicamente 5 actores principales, respetando el orden en el que vienen.

In [12]:
type(df_credits['cast'][0]) # se verifica el tipo de datos, deben ser listas o dicccionarios para poder normalizarlos

str

In [13]:
#Se convierte el tipo de datos de String a Diccionario o lista

def convertir_a_diccionario(cadena):
    try:
        return ast.literal_eval(cadena)
    except (ValueError, SyntaxError):
        return None  # O cualquier valor predeterminado para manejar errores

# Aplicar la conversión a la columna
df_credits['cast'] = df_credits['cast'].apply(convertir_a_diccionario)

type(df_credits['cast'][0])

list

In [15]:
# Se muestran los tipos de datos únicos en la columnas

df_credits['cast'].apply(type).unique()

array([<class 'list'>], dtype=object)

In [16]:
# Crear la función normalize_entry para procesar y normalizar diferentes tipos de datos en una entrada dentro de un DataFrame (diccionarios y listas de diccionarios)
def normalize_entry(entry):
    if isinstance(entry, dict):
        return pd.json_normalize(entry)
    elif isinstance(entry, list) and all(isinstance(i, dict) for i in entry):
        return pd.json_normalize(entry)
    else:
        return pd.DataFrame()  # Retorna un DataFrame vacío para valores que no son diccionarios ni listas de diccionarios

# Se crea una lista para almacenar los DataFrames expandidos
expanded_dfs = []

# Iterar sobre cada fila del DataFrame original
for idx, row in df_credits.iterrows():
    entry = row['cast']
    normalized_df = normalize_entry(entry)
    normalized_df['id'] = row['id']  # Añadir la columna 'id'
    normalized_df['order'] = range(1, len(normalized_df) + 1)  # Añadir la columna 'order'
    expanded_dfs.append(normalized_df)

# Concatenar todos los DataFrames expandidos en uno solo
df_cast = pd.concat(expanded_dfs, ignore_index=True)

In [17]:
df_cast.columns #Para verificar las columnas existentes en el nuevo data frame

Index(['cast_id', 'character', 'credit_id', 'gender', 'id', 'name', 'order',
       'profile_path'],
      dtype='object')

In [18]:
df_cast.shape #Para verificar el tamaño del nuevo data frame

(562132, 8)

In [20]:
df_cast = df_cast[['id'] + [col for col in df_cast.columns if col != 'id']] #Dejo de primera la columna "id"
df_cast = df_cast.drop(columns=['credit_id', 'gender', 'cast_id', 'profile_path']) #Elimino las columnas innecesarias

In [23]:
df_cast = df_cast[df_cast['order'].between(1, 5)] # Filtrar los primeros 5 actores y eliminar el resto

In [28]:
df_cast.tail(20)

,id,character,name,order
562096,30840,Sir Miles Folcanet,Jürgen Prochnow,4
562097,30840,Baron Roger Daguerre,Jeroen Krabbé,5
562098,439050,,Leila Hatami,1
562099,439050,,Kourosh Tahami,2
562100,439050,,Elham Korda,3
562101,111109,Sister Angela,Angel Aquino,1
562102,111109,Homer,Perry Dizon,2
562103,111109,Crazy Woman/Virgin,Hazel Orencio,3
562104,111109,Amang Tiburcio,Joel Torre,4
562105,111109,Ex-convict/Dindo,Bart Guingona,5


In [26]:
df_cast.shape #Para verificar el tamaño nuevamente

(202338, 4)

In [161]:
df_cast['order'] = df_cast['order'].astype(str)  # Para que la columna order se maneje como cadena
df_cast = df_cast.pivot_table(index=['id'], columns='order', values=['character', 'name'], aggfunc='first') # se hace  el pivot de la tabla
df_cast.columns = [f"{val}_{col}" for val, col in df_cast.columns] # Ajustamos el formato del DataFrame resultante
df_cast.reset_index(inplace=True)

In [162]:
df_cast.head()

,id,character_1,character_2,character_3,character_4,character_5,name_1,name_2,name_3,name_4,name_5
0,2,Taisto Olavi Kasurinen,Irmeli Katariina Pihlaja,Mikkonen,Riku,NaN,Turo Pajala,Susanna Haavisto,Matti Pellonpää,Eetu Hilkamo,NaN
1,3,Nikander,Ilona Rajamäki,Melartin,Co-worker,Ilona's Girlfriend,Matti Pellonpää,Kati Outinen,Sakari Kuosmanen,Esko Nikkari,Kylli Köngäs
2,5,Ted the Bellhop,Man,Angela,Elspeth,Margaret,Tim Roth,Antonio Banderas,Jennifer Beals,Madonna,Marisa Tomei
3,6,Frank Wyatt,Mike Peterson,Fallon,Ray Cochran,Sykes,Emilio Estevez,Cuba Gooding Jr.,Denis Leary,Jeremy Piven,Peter Greene
4,11,Luke Skywalker,Han Solo,Princess Leia Organa,Grand Moff Tarkin,"Obi-Wan ""Ben"" Kenobi",Mark Hamill,Harrison Ford,Carrie Fisher,Peter Cushing,Alec Guinness


In [163]:
df_cast.shape

(43018, 11)

In [164]:
# Calcular el uso de memoria del DataFrame en bytes
uso_memoria = df_cast.memory_usage(deep=True).sum()

# Convertir el uso de memoria a megabytes
uso_memoria_mb = uso_memoria / (1024 * 1024)

print(f"El DataFrame utiliza aproximadamente {uso_memoria_mb:.2f} MB de memoria.")


El DataFrame utiliza aproximadamente 28.26 MB de memoria.
